In [70]:
import pandas as pd
from ftplib import FTP
from datetime import datetime
import time
import numpy as np
import pymongo
from pymongo import MongoClient

from geopy import distance

import get_recent_days as gtdys
import muni_etl
import labelling as lblng

In [101]:
client = MongoClient('localhost', 27017)

db = client['testavl']
in_coll = db['clean']
out_coll = db['clean_out']

In [105]:
labeler = lblng.Labeling(in_coll, out_coll)

In [108]:
labeler.sched_trps['departure_time'].min()

'05:48:00'

In [104]:
labeler.label_single_starts()

483019   1900-01-01 20:50:00
483327   1900-01-01 18:30:00
483635   1900-01-01 16:10:00
483943   1900-01-01 13:50:00
484287   1900-01-01 11:30:00
484507   1900-01-01 05:50:00
484639   1900-01-01 07:20:00
484815   1900-01-01 09:20:00
484930   1900-01-01 22:34:00
Name: departure_time, dtype: datetime64[ns]
483151   1900-01-01 19:50:00
483459   1900-01-01 17:30:00
483767   1900-01-01 15:10:00
484075   1900-01-01 12:50:00
484419   1900-01-01 10:30:00
484595   1900-01-01 06:50:00
484727   1900-01-01 08:20:00
Name: departure_time, dtype: datetime64[ns]


ValueError: time data '25:14:00' does not match format '%H:%M:%S'